# Lab 06       
## Student ID:  3606213
## Name:        Jordan Richard

In [11]:
import math
import time

cities = [{"x": 38.24, "y": 20.42},
          {"x": 39.57, "y": 26.15},
          {"x": 40.56, "y": 25.32},
          {"x": 36.26, "y": 23.12},
          {"x": 33.48, "y": 10.54},
          {"x": 37.56, "y": 12.19},
          {"x": 38.42, "y": 13.11},
          {"x": 37.52, "y": 20.44},
          {"x": 41.23, "y": 9.10},
          {"x": 41.17, "y": 13.05},
          {"x": 36.08, "y": -5.21},
          {"x": 38.47, "y": 15.13},
          {"x": 38.15, "y": 15.35},
          {"x": 37.51, "y": 15.17},
          {"x": 35.49, "y": 14.32},
          {"x": 39.36, "y": 19.56}]

opt_old = [1, 14, 13, 12, 7, 6, 15, 5, 11, 9, 10, 16, 3, 2, 4, 8]
opt = [x - 1 for x in opt_old]


def dist(p,q):
    dx = p['x'] - q['x']
    dy = p['y'] - q['y']
    return math.sqrt(dx*dx + dy*dy)

def travel_distance(path,cities):
    p = cities[path[0]]
    d = 0
    for n in path[1:]:
        q = cities[n]
        d += dist(p,q)
        p = q
    d += dist(p,cities[path[0]])
    return d

class Node:
    def __init__(self, other):
        if other is None:
            self.cur = 0
            self.visited = [False for i in range(16)]
            self.path = [-1 for i in range(16)]
        else:
            self.cur = other.cur
            self.visited = other.visited[:]
            self.path = other.path[:]
    def __repr__(self):
        cost = travel_distance(self.path, cities)
        return "{c:4.2f}:{p}".format(p=self.path,
                                     c=cost)
    def cost(self):
        if self.cur < 1:
            return 0
        else:
            p = cities[self.path[0]]
            d = 0
            for n in self.path[1:self.cur]:
                q = cities[n]
                d += dist(p,q)
                p = q
            return d
    def heuristic(self,city):
        if self.cur < 1:
            return 0
        else:
            d1 = dist(cities[self.path[self.cur - 1]],cities[city])
            d2 = dist(cities[city],cities[self.path[0]])
            return d1 + d2
    def visit(self, city):
        self.visited[city] = True
        self.path[self.cur] = city
        self.cur += 1
    def feasible(self):
        return self.cur == 16


class MyException(Exception):
    def __init__(self,val):
        self.val = val

class TravelAgent:
    def __init__(self, limit):
        self.seq = []
        self.limit = limit
    def update_state(self, percept):
        pass
    def formulate_goal(self):
        # based on state
        return None
    def formulate_problem(self):
        # based on state and goal
        self.state = Node(None)
    def ids(self,parent,limit,depth):
        if parent.feasible():
            self.state = parent
            raise MyException(parent)
        else:
            for i in range(16):
                n = Node(parent)
                if not n.visited[i]:
                    next_city = i
                    d1 = n.cost()
                    d2 = n.heuristic(next_city)
                    d = d1 + d2
                    if d < limit:
                        n.visit(next_city)
                        self.ids(n, limit,depth+1)

    def search(self):
        # based on state
        try:
            print("Exception")
            self.ids(self.state, self.limit,0)
        except MyException:
            pass
        if self.state.feasible():
            return [self.state]
        else:
            return None

    def recommendation(self):
        if self.seq:
            return self.seq[0]
        else:
            return None

    def remainder(self):
        if self.seq:
            self.seq = self.seq[1:]
        
    def action(self, percept):
        self.state = self.update_state(percept)
        self.goal = self.formulate_goal()
        self.problem = self.formulate_problem()
        if self.seq == []:
            self.seq = self.search()
        action = self.recommendation()
        seq = self.remainder()
        return action
            
def test():
    A = TravelAgent(90)

    tm = time.time_ns()
    print(A.action(None))
    print('elapsed: ', time.time_ns() - tm)

    B = TravelAgent(85)

    tm = time.time_ns()
    print(B.action(None))
    print('elapsed: ', time.time_ns() - tm)

    C = TravelAgent(82)

    tm = time.time_ns()
    print(C.action(None))
    print('elapsed: ', time.time_ns() - tm)
            
test()

Exception
89.56:[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 14, 11, 12, 13, 15, 7]
elapsed:  694772587
Exception
84.86:[0, 1, 2, 3, 4, 5, 6, 9, 8, 10, 14, 11, 12, 13, 7, 15]
elapsed:  1169416776
Exception
81.42:[0, 1, 2, 3, 4, 10, 8, 5, 6, 9, 11, 12, 13, 14, 7, 15]
elapsed:  163429350653


# Answer 1(a)

Three agents are created, named A, B, and C. Each is initialized with a different depth limit, 90, 85, and 82 respectively.

# Answer 1(b)

The heuristic simply adds the distance to the next city to the current city, ignoring all others.

# Answer 1(c)

By adding limits to the search operation, we can limit our search from visiting nodes too deep into the tree. This way we can modify our search to return an acceptable (but not necessarily optimal) solution within a reasonable amount of time.

# Answer 1(d)

The Agent 'C' took the longest time to finish executing due to its more restrictive limit. Because of this, the travel agent had to search for longer and for more subtrees until a solution satisfying this more restrictive limit was found.

# Answer 1(e)

By using exceptions,  we are given more control over execution and are able to continue or stop it depending on if a desired condition is met (or not). In this case, we were able to stop execution of our search once a desired solution was found rather than continuing a pointless search over the entire tree.

# Answer 2(a)

# Answer 2(b)

* find the distance to the city
* find the distance to the starting city
  * this heuristic ignores all other visits
  * very simple but admissible
* add the two

# Answer 2(c)

| limit | elapsed for old | elapsed for new |
| --- | --- | --- |
|  82   |     xyz         |      xyz        |
